In [3]:
import pandas as pd
import numpy as np
import os

# ================= 文件名配置 =================
# 请确保这些文件都在你的目录下
file_base_panel = 'final_panel_data_1990_2019.csv'    # 之前生成的氮投入+气象面板
file_yield = '25F121B9-3EB4-3BE7-AC50-870ADF7D8454.csv' # 最早上传的产量原始文件
file_gas = 'PCU2111112111113.csv'                    # 天然气价格
file_biotech = 'BiotechCropsAllTables2024.csv'       # 转基因数据
output_file = 'final_dataset_for_regression.csv'     # 最终输出文件
# ============================================

print(">>> 正在读取文件...")

# 1. 读取基础面板 (Ninput + Weather)
try:
    df_panel = pd.read_csv(file_base_panel, encoding='utf-8')
except:
    df_panel = pd.read_csv(file_base_panel, encoding='ISO-8859-1')

# 2. 读取并清洗产量数据 (Yield) - 之前可能漏掉了这个合进面板
print(">>> 正在处理产量数据...")
try:
    df_yield_raw = pd.read_csv(file_yield)
except:
    df_yield_raw = pd.read_csv(file_yield, encoding='ISO-8859-1')

# 生成 ANSI Code 用于匹配
def get_ansi(row):
    try:
        # 确保是 5 位字符串 (例如 17107)
        if pd.notna(row['State ANSI']) and pd.notna(row['County ANSI']):
            return f"{int(row['State ANSI']):02d}{int(row['County ANSI']):03d}"
    except:
        pass
    return None

df_yield_raw['ANSI Code'] = df_yield_raw.apply(get_ansi, axis=1)
# 提取需要的列：年份、ANSI码、产量值
df_yield_clean = df_yield_raw[['Year', 'ANSI Code', 'Value']].dropna()
df_yield_clean.rename(columns={'Value': 'Yield_BuPerAcre'}, inplace=True)

# 3. 读取天然气数据 (IV)
print(">>> 正在处理天然气数据...")
df_gas = pd.read_csv(file_gas)
df_gas['observation_date'] = pd.to_datetime(df_gas['observation_date'])
df_gas['Year'] = df_gas['observation_date'].dt.year
# 计算年均价格
df_gas_annual = df_gas.groupby('Year')['PCU2111112111113'].mean().reset_index()
# 【修正】这里修复了你之前的报错
df_gas_annual.rename(columns={'PCU2111112111113': 'Natural_Gas_Price'}, inplace=True)

# 4. 读取转基因数据 (Control)
print(">>> 正在处理转基因数据...")
try:
    df_biotech = pd.read_csv(file_biotech, encoding='ISO-8859-1')
except:
    df_biotech = pd.read_csv(file_biotech, encoding='cp1252')

# 清洗列名
df_biotech.columns = [c.strip() for c in df_biotech.columns]
target_attr = 'All GE varieties (percent of all corn planted)'
df_bio = df_biotech[
    (df_biotech['Attribute'] == target_attr) & 
    (df_biotech['State'] != 'United States')
].copy()

df_bio['State'] = df_bio['State'].str.upper().str.strip()
# 清洗数值
df_bio['Biotech_Rate'] = pd.to_numeric(df_bio['Value'], errors='coerce')
df_bio = df_bio[['State', 'Year', 'Biotech_Rate']]

# ================= 合并所有数据 =================
print(">>> 正在合并所有变量...")

# 确保关联键格式一致
df_panel['ANSI Code'] = df_panel['ANSI Code'].astype(str).str.zfill(5)
df_yield_clean['ANSI Code'] = df_yield_clean['ANSI Code'].astype(str).str.zfill(5)
df_panel['State'] = df_panel['State'].str.upper().str.strip()

# A. 合并产量 (按 ANSI Code 和 Year)
df_final = pd.merge(df_panel, df_yield_clean, on=['ANSI Code', 'Year'], how='inner')

# B. 合并天然气 (按 Year)
df_final = pd.merge(df_final, df_gas_annual, on='Year', how='left')

# C. 合并转基因 (按 State 和 Year)
df_final = pd.merge(df_final, df_bio, on=['State', 'Year'], how='left')

# 填补 2000 年前的转基因数据为 0
df_final.loc[df_final['Year'] < 2000, 'Biotech_Rate'] = 0

# 保存
df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n成功！最终数据集已保存为: {output_file}")
print("包含列:", df_final.columns.tolist())

>>> 正在读取文件...
>>> 正在处理产量数据...
>>> 正在处理天然气数据...
>>> 正在处理转基因数据...
>>> 正在合并所有变量...

成功！最终数据集已保存为: final_dataset_for_regression.csv
包含列: ['Year', 'State', 'County', 'crop_type', 'Area (Mha)', 'Ninput (kg N/ha/yr)', 'NUE (kg N/kg N)', 'Recovered N (kg N/ha/yr)', 'Nsurplus (kg N/ha/yr)', 'ppt (inches)', 'tmin (degrees F)', 'tmean (degrees F)', 'tmax (degrees F)', 'ANSI Code', 'Yield_BuPerAcre', 'Natural_Gas_Price', 'Biotech_Rate']
